In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import StratifiedKFold

from utilities import load_data

In [7]:
np.random.seed(0)
X_high,y_high = load_data(100, amplitude=10)
X, y = load_data(200, amplitude=1)
X_low = X[200:]
y_low = y[200:]

In [19]:
skfold = StratifiedKFold(n_splits=5)

acc_per_fold_high = []
acc_per_fold_low = []
loss_per_fold = []

for train, test in skfold.split(X_high,y_high):

    #define the model
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(32, (3, 3), activation='tanh', input_shape=X.shape[1:]))
    model.add(keras.layers.AveragePooling2D((3, 3)))
    
    model.add(keras.layers.Conv2D(16, (3, 3), activation='tanh'))
    model.add(keras.layers.AveragePooling2D((2, 2)))
    
    model.add(keras.layers.Conv2D(8, (3, 3), activation='tanh'))
    model.add(keras.layers.AveragePooling2D((2, 2)))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    if len(acc_per_fold_high)==0 : model.summary()
    #compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    #fit the model
    history = model.fit(X_high[train], y_high[train], epochs=10, batch_size=25, verbose=1)

    #evaluate the model
    print('\nEvaluation of the model : \n')
    print('Train set : \n')
    model.evaluate(X_high[train], y_high[train])
    print('Test set(10**-6) : \n')
    scores_high = model.evaluate(X_high[test], y_high[test])
    print('Test set(10**-7) : \n')
    scores_low = model.evaluate(X_low[test], y_low[test])
    
    acc_per_fold_high.append(scores_high[1] * 100)
    acc_per_fold_low.append(scores_low[1] * 100)

    print("\n-----------------------------------\n")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 398, 398, 32)      320       
_________________________________________________________________
average_pooling2d_6 (Average (None, 132, 132, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 130, 130, 16)      4624      
_________________________________________________________________
average_pooling2d_7 (Average (None, 65, 65, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 63, 63, 8)         1160      
_________________________________________________________________
average_pooling2d_8 (Average (None, 31, 31, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 7688)             